# Ejemplo Simpy simulador de procesos

## Este ejemplo modela la caja de un banco y clientes que llegan T veces al azar. Cada cliente tiene una cierta paciencia. Cuando uno llega a caja se queda ahi un tiempo antes de irse.

In [1]:

import random
import simpy


RANDOM_SEED = 40  # Semilla randomica
NUEVOS_CLIENTES = 5  # Numero total de clientes
INTERVALO_CLIENTES = 10.0  # Genera nuevos clientes aproximadamente cada x segundos
PACIENCIA_MINIMA = 1  # Paciencia minima del cliente
PACIENCIA_MAXIMA = 3  # Paciencia maxima del cliente


def fuente(env, number, interval, counter):
    """Aqui se generan clientes al azar"""
    for i in range(number):
        c = cliente(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def cliente(env, name, counter, time_in_bank):
    """Un cliente llega, es atendido y se va."""
    arrive = env.now
    print('%7.4f %s: Aqui estoy' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(PACIENCIA_MINIMA, PACIENCIA_MAXIMA)
        # Espera llegar a la caja o si se acaba la paciencia se va
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # llega a la caja
            print('%7.4f %s: Esperó %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Terminó' % (env.now, name))

        else:
            # Se enoja y se va
            print('%7.4f %s: Se Enojó despues de %6.3f' % (env.now, name, wait))


# Iniciamos la simulación
print('Inicio de la simulación')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Iniciamos el proceso
counter = simpy.Resource(env, capacity=1)
env.process(fuente(env, NUEVOS_CLIENTES, INTERVALO_CLIENTES, counter))
env.run()

Inicio de la simulación
 0.0000 Customer00: Aqui estoy
 0.0000 Customer00: Esperó  0.000
 0.3884 Customer00: Terminó
 6.1361 Customer01: Aqui estoy
 6.1361 Customer01: Esperó  0.000
 9.4549 Customer02: Aqui estoy
10.8288 Customer03: Aqui estoy
11.1517 Customer02: Se Enojó despues de  1.697
12.7103 Customer03: Se Enojó despues de  1.882
19.2352 Customer01: Terminó
32.0106 Customer04: Aqui estoy
32.0106 Customer04: Esperó  0.000
33.6800 Customer04: Terminó


### Fuente: Basado en el programa bank08.py del tutorial de Simpy 2 TheBank (KGM)